# Train Legal Classifiers

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
from johnsnowlabs import * 

/databricks/python/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
 warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
 🚨 Your Spark-Healthcare is outdated, installed==4.2.0 but latest version==4.2.0
You can run jsl.install() to update Spark-Healthcare

# Multilabel classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Legal/data/finance_data.csv

dbutils.fs.cp("file:/databricks/driver/finance_data.csv", "dbfs:/") 

Out[2]: True

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_data.csv')
df['label'] = df['label'].apply(eval)

In [0]:
data = spark.createDataFrame(df)

# If you have a single dataset, then split it or else you can load the test dataset the same way that you load the train data.
train, test = data.randomSplit([0.8, 0.2], seed = 123)

In [0]:
train.show(truncate=50)

+--------------------------------------------------+-----------------------------+
 provision| label|
+--------------------------------------------------+-----------------------------+
(a) Consultant or Company may terminate this Pr...| [terminations]|
(a) Effective as of the Effective Date, the Hol...| [waivers, terminations]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Agent or any Len...| [waivers, amendments]|
(a) No failure or delay of the Administrative A...| [waivers, amendments]|
(a) The Credit Agreement is, effective as of th...| [amendments]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The provisions of this Agreement shall be ...| [assigns, successors]|
(a) All of the representations and warranties m...|[representations, warranties]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Collateral Agent...| [waivers, amendments]|
(a) No failure or delay of the Agents or any Le...| [waivers, amendments]|
(a) THIS AGREEMENT AND EACH OTHER LOAN DOCUMENT...| [governing laws]|
(a) THIS AGREEMENT WAS NEGOTIATED IN THE STATE ...| [governing laws]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) This Agreement may be amended, supplemented...| [waivers, amendments]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The Performance Guarantor hereby waives pr...| [waivers]|
+--------------------------------------------------+-----------------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 [governing laws]| 727|
 [counterparts]| 569|
 [notices]| 556|
 [entire agreements]| 556|
 [severability]| 496|
 [survival]| 358|
[assigns, success...| 319|
 [amendments]| 319|
 [expenses]| 248|
 [terminations]| 224|
[waivers, amendme...| 198|
[representations,...| 191|
 [waivers]| 186|
 [assignments]| 186|
 [representations]| 97|
[amendments, enti...| 73|
 [successors]| 64|
 [warranties]| 40|
[amendments, term...| 27|
[assignments, suc...| 10|
+--------------------+-----+
only showing top 20 rows

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
      .setInputCol("provision") \
      .setOutputCol("document") \
      .setCleanupMode("shrink")

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classsifierdl = nlp.MultiClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(30)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 656 ms, sys: 195 ms, total: 851 ms
Wall time: 2min 55s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','provision',"class.result").toPandas()
preds_df.head()

Out[12]:

,label,provision,result
0,"[waivers, amendments]",(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]"
1,[assignments],"(a) Seller, the Agent, each Managing Agent, ea...","[successors, assignments]"
2,[waivers],(a) Any provision of this Agreement may be wai...,"[waivers, amendments]"
3,[notices],(a) Except where telephonic instructions or no...,[notices]
4,"[governing laws, entire agreements]","(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]"


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


Classification report: 
 precision recall f1-score support

 0 0.88 0.79 0.83 643
 1 0.70 0.61 0.65 210
 2 0.79 0.87 0.83 333
 3 0.99 0.98 0.98 579
 4 0.97 0.94 0.96 667
 5 0.95 0.96 0.95 249
 6 0.98 0.98 0.98 747
 7 0.95 0.96 0.96 557
 8 0.86 0.81 0.83 288
 9 0.96 0.94 0.95 515
 10 0.86 0.89 0.87 407
 11 0.93 0.90 0.92 363
 12 0.84 0.79 0.82 256
 13 0.83 0.83 0.83 401
 14 0.82 0.75 0.78 231

 micro avg 0.91 0.89 0.90 6446
 macro avg 0.89 0.87 0.88 6446
weighted avg 0.91 0.89 0.90 6446
 samples avg 0.89 0.90 0.89 6446

F1 micro averaging: 0.9005014102162331
ROC: 0.9420396968437851

## With RoBerta Embeddings

We do not have have any specific Legal Sentence Embeddings, but we can use Legal RoBerta Embeddings and then average them.

In [0]:
embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("provision") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = nlp.MultiClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(1)\
    .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 13.4 s, sys: 6.13 s, total: 19.5 s
Wall time: 1h 7min 1s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('provision','label',"class.result").toPandas()

In [0]:
preds_df.head()

Out[20]:

,provision,label,result
0,(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]",[waivers]
1,"(a) Seller, the Agent, each Managing Agent, ea...",[assignments],"[successors, assigns]"
2,(a) Any provision of this Agreement may be wai...,[waivers],[]
3,(a) Except where telephonic instructions or no...,[notices],[notices]
4,"(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]","[governing laws, entire agreements]"


In [0]:
import os
log_file_name = os.listdir("/root/annotator_logs")[0]

with open("/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training started - epochs: 6 - learning_rate: 0.001 - batch_size: 64 - training_examples: 22042 - classes: 15
Epoch 0/6 - 6.60s - loss: 0.096051626 - acc: 0.9705064 - batches: 345
Epoch 1/6 - 4.51s - loss: 0.039551556 - acc: 0.98879695 - batches: 345
Epoch 2/6 - 4.74s - loss: 0.03486474 - acc: 0.9903383 - batches: 345
Epoch 3/6 - 4.52s - loss: 0.0321748 - acc: 0.99127585 - batches: 345
Epoch 4/6 - 4.64s - loss: 0.030249653 - acc: 0.99202967 - batches: 345
Epoch 5/6 - 4.48s - loss: 0.028734822 - acc: 0.9925955 - batches: 345



In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


Classification report: 
 precision recall f1-score support

 0 0.95 0.62 0.75 643
 1 0.85 0.30 0.44 210
 2 0.75 0.90 0.82 333
 3 0.97 0.99 0.98 579
 4 0.98 0.96 0.97 667
 5 0.96 0.97 0.97 249
 6 0.98 0.99 0.99 747
 7 0.97 0.96 0.96 557
 8 0.91 0.59 0.72 288
 9 0.99 0.97 0.98 515
 10 0.84 0.93 0.88 407
 11 0.94 0.83 0.88 363
 12 0.93 0.67 0.78 256
 13 0.95 0.67 0.79 401
 14 0.83 0.72 0.77 231

 micro avg 0.94 0.85 0.89 6446
 macro avg 0.92 0.81 0.85 6446
weighted avg 0.94 0.85 0.88 6446
 samples avg 0.87 0.86 0.86 6446

F1 micro averaging: 0.8881101784695624
ROC: 0.920202035468987

### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

Out[22]: [DocumentAssembler_0576ba3dfdd3,
 REGEX_TOKENIZER_85da7f5aa645,
 ROBERTA_EMBEDDINGS_b915dff90901,
 SentenceEmbeddings_536bd4fb8757,
 MultiClassifierDLModel_9bb76951c013]

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/MultilabelClfRoBerta')

In [0]:
# Load back  saved Multilabel Classifier Model
MultilabelClfModel = nlp.MultiClassifierDLModel.load('/dbfs/MultilabelClfRoBerta')

In [0]:
ld_pipeline = Pipeline(stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, MultilabelClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("provision"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('provision','label',"class.result").toPandas()

In [0]:
ld_preds_df.head(10)

Out[34]:

,provision,label,result
0,(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]",[waivers]
1,"(a) Seller, the Agent, each Managing Agent, ea...",[assignments],"[successors, assigns]"
2,(a) Any provision of this Agreement may be wai...,[waivers],[]
3,(a) Except where telephonic instructions or no...,[notices],[notices]
4,"(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]","[governing laws, entire agreements]"
5,(a) This Agreement may be executed by one or m...,[counterparts],[counterparts]
6,A Any notice which either party hereto may be ...,[notices],[notices]
7,A counterpart original of this Amendment duly ...,[amendments],[counterparts]
8,Advisor represents that Advisor’s services und...,[representations],[]
9,"After the Termination Date, this Agreement sha...",[terminations],"[survival, terminations]"


# Multiclass classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/legal/data/finance_clf_data.csv

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2232641861142586> in <module> 
 1 get_ipython ( ) . system ( ' wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/finance/data/finance_clf_data.csv' ) 
 ----> 2 dbutils . fs . cp ( "file:/databricks/driver/finance_clf_data.csv" , "dbfs:/" ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 379 exc . __context__ = None 
 380 exc . __cause__ = None 
 --> 381 raise exc
 382 
 383 return f_with_exception_handling

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: File file:/databricks/driver/finance_clf_data.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:428)
	at com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem.super$getFileStatus(WorkspaceLocalFileSystem.scala:67)
	at com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem.$anonfun$getFileStatus$1(WorkspaceLocalFileSystem.scala:67)
	at com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem.$anonfun$getFileStatus$1$adapted(WorkspaceLocalFileSystem.scala:66)
	at com.databricks.backend.daemon.driver.WSFSCredentialForwardingHelper.$anonfun$withWSFSCredentials$1(WorkspaceLocalFileSystem.scala:180)
	at com.databricks.backend.daemon.driver.WSFSCredentialForwardingHelper.withWSFSCredentials(WorkspaceLocalFileSystem.scala:151)
	at com.databricks.backend.daemon.driver.WSFSCredentialForwardingHelper.withWSFSCredentials(WorkspaceLocalFileSystem.scala:180)
	at com.databricks.backend.daemon.driver.WSFSCredentialForwardingHelper.withWSFSCredentials$(WorkspaceLocalFileSystem.scala:178)
	at com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem.withWSFSCredentials(WorkspaceLocalFileSystem.scala:30)
	at com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem.getFileStatus(WorkspaceLocalFileSystem.scala:66)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$withCpSafetyChecks$2(DBUtilsCore.scala:101)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$withCpSafetyChecks$1(DBUtilsCore.scala:98)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withCpSafetyChecks(DBUtilsCore.scala:98)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cp(DBUtilsCore.scala:177)
	at com.databricks.backend.daemon.dbutils.FSUtils.cp(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_clf_data.csv')

In [0]:
df.head()

Out[46]:

,text,label,len
0,Presently we do not believe any U S or State r...,business,402
1,\r\nnetwork outages or performance degradatio...,risk_factors,496
2,Available Information\r\nOur reports filed wit...,business,356
3,\r\n 42 530\r\n \r\n \r\n \r\n \r\n \r\n 42 5...,financial_statements,359
4,8\r\nTable of Contents\r\ndevelopment employee...,business,582


In [0]:
df['label'].value_counts()

Out[47]: risk_factors 1926
financial_statements 1888
business 970
financial_conditions 346
form_10k_summary 240
executives_compensation 155
controls_procedures 138
equity 111
market_risk 100
executives 73
legal_proceedings 51
properties 48
security_ownership 46
exhibits 36
Name: label, dtype: int64

In [0]:
data = spark.createDataFrame(df)

train, test = data.randomSplit([0.8, 0.2], seed = 100)

In [0]:
from pyspark.sql.functions import col

train.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 1529|
financial_statements| 1491|
 business| 769|
financial_conditions| 267|
 form_10k_summary| 196|
executives_compen...| 128|
 controls_procedures| 107|
 equity| 92|
 market_risk| 75|
 executives| 58|
 properties| 39|
 legal_proceedings| 36|
 security_ownership| 34|
 exhibits| 31|
+--------------------+-----+

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 397|
financial_statements| 397|
 business| 201|
financial_conditions| 79|
 form_10k_summary| 44|
 controls_procedures| 31|
executives_compen...| 27|
 market_risk| 25|
 equity| 19|
 executives| 15|
 legal_proceedings| 15|
 security_ownership| 12|
 properties| 9|
 exhibits| 5|
+--------------------+-----+

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document") 

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classsifierdl = legal.ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(30)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
clf_pipelineModel = clf_pipeline.fit(train)

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()
preds_df.head()

Out[49]:

,label,text,result
0,risk_factors,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,[financial_statements]
1,financial_statements,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,[financial_statements]
2,form_10k_summary,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,[financial_statements]
3,financial_statements,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,[financial_statements]
4,risk_factors,\r\n \r\nAn assertion by a third party that w...,[risk_factors]


In [0]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

precision recall f1-score support

 business 0.71 0.77 0.74 201
 controls_procedures 0.00 0.00 0.00 31
 equity 0.00 0.00 0.00 19
 executives 0.00 0.00 0.00 15
executives_compensation 0.00 0.00 0.00 27
 exhibits 0.00 0.00 0.00 5
 financial_conditions 0.00 0.00 0.00 79
 financial_statements 0.63 0.93 0.75 397
 form_10k_summary 0.00 0.00 0.00 44
 legal_proceedings 0.00 0.00 0.00 15
 market_risk 0.00 0.00 0.00 25
 properties 0.00 0.00 0.00 9
 risk_factors 0.74 0.88 0.81 397
 security_ownership 0.00 0.00 0.00 12

 accuracy 0.69 1276
 macro avg 0.15 0.18 0.16 1276
 weighted avg 0.54 0.69 0.60 1276

### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

Out[52]: [DocumentAssembler_0de333ac0914,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 LegalClassifierDLModel_f1c013357b5e]

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/Clf_Use')

In [0]:
# Load back  saved Classifier Model
ClfModel = legal.ClassifierDLModel.load('/dbfs/Clf_Use')

In [0]:
ld_pipeline = Pipeline(stages=[document_assembler, embeddings,ClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('text','label',"class.result").toPandas()

In [0]:
ld_preds_df.head()

Out[59]:

,text,label,result
0,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,risk_factors,[financial_statements]
1,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,financial_statements,[financial_statements]
2,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,form_10k_summary,[financial_statements]
3,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,financial_statements,[financial_statements]
4,\r\n \r\nAn assertion by a third party that w...,risk_factors,[risk_factors]


## With RoBerta Embeddings

We do not have Legal Sentence Embeddings yet, But we can use the Legal RoBerta Embeddings and then average them.

In [0]:
embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = legal.ClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(8)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
clf_pipelineModel = clf_pipeline.fit(train)

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()

In [0]:
preds_df.head()

Out[68]:

,label,text,result
0,risk_factors,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,[financial_statements]
1,financial_statements,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,[financial_statements]
2,form_10k_summary,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,[financial_statements]
3,financial_statements,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,[financial_statements]
4,risk_factors,\r\n \r\nAn assertion by a third party that w...,[risk_factors]


In [0]:
log_files = os.listdir("/root/annotator_logs")

with open("/root/annotator_logs/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 8 - learning_rate: 0.001 - batch_size: 64 - training_examples: 9783 - classes: 14
Epoch 0/8 - 1.31s - loss: 352.6541 - acc: 0.57318145 - batches: 153
Epoch 1/8 - 1.09s - loss: 348.1564 - acc: 0.5952826 - batches: 153
Epoch 2/8 - 1.09s - loss: 347.85394 - acc: 0.596105 - batches: 153
Epoch 3/8 - 1.08s - loss: 347.59656 - acc: 0.5982637 - batches: 153
Epoch 4/8 - 1.06s - loss: 347.57974 - acc: 0.59982246 - batches: 153
Epoch 5/8 - 1.11s - loss: 347.59897 - acc: 0.600542 - batches: 153
Epoch 6/8 - 1.10s - loss: 347.61884 - acc: 0.6012784 - batches: 153
Epoch 7/8 - 1.07s - loss: 347.6311 - acc: 0.601998 - batches: 153



In [0]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


                         precision    recall  f1-score   support

               business       0.00      0.00      0.00       420
    controls_procedures       0.00      0.00      0.00        51
                 equity       0.00      0.00      0.00        49
             executives       0.00      0.00      0.00        39
executives_compensation       0.00      0.00      0.00        80
               exhibits       0.00      0.00      0.00        15
   financial_conditions       0.00      0.00      0.00       105
   financial_statements       0.58      0.96      0.72       743
       form_10k_summary       0.00      0.00      0.00       106
      legal_proceedings       0.00      0.00      0.00        22
            market_risk       0.00      0.00      0.00        46
             properties       0.00      0.00      0.00        24
           risk_factors       0.59      0.97      0.73       760
     security_ownership       0.00      0.00      0.00        14

               accuracy

# Save model and Zip it for Modelshub Upload/Downloads

In [0]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('ClfBert')

# cd into saved dir and zip
! cd /content/ClfBert ; zip -r /content/ClfBert.zip *

  adding: classifierdl_tensorflow (deflated 58%)
  adding: fields/ (stored 0%)
  adding: fields/datasetParams/ (stored 0%)
  adding: fields/datasetParams/.part-00003.crc (stored 0%)
  adding: fields/datasetParams/.part-00002.crc (stored 0%)
  adding: fields/datasetParams/part-00000 (deflated 27%)
  adding: fields/datasetParams/.part-00000.crc (stored 0%)
  adding: fields/datasetParams/part-00002 (deflated 27%)
  adding: fields/datasetParams/._SUCCESS.crc (stored 0%)
  adding: fields/datasetParams/part-00003 (deflated 32%)
  adding: fields/datasetParams/part-00001 (deflated 26%)
  adding: fields/datasetParams/.part-00001.crc (stored 0%)
  adding: fields/datasetParams/_SUCCESS (stored 0%)
  adding: metadata/ (stored 0%)
  adding: metadata/part-00000 (deflated 39%)
  adding: metadata/.part-00000.crc (stored 0%)
  adding: metadata/._SUCCESS.crc (stored 0%)
  adding: metadata/_SUCCESS (stored 0%)
